# Generating Ledger Features and Exploring Corrleation with Closetime

## Step 1. Generating Ledger Features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import preprocessing
#from scipy.stats import shapiro
from scipy.cluster.hierarchy import dendrogram, linkage 
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from google.cloud import bigquery
from google.oauth2 import service_account

|LedgerIndex|Sum of XRP Volume|# of transactions in the ledger|Sum of total fees| Number of OfferCreates|Number of Payments|Number of Account Sets| Number of OfferCancels|
|-----|-------|------|------|-----|------|-----|------|



In [5]:
xrp2=pd.read_csv('../../Data/12_1_2019_to_12_30_2019_DownselectALL.csv')

In [3]:
xrp2

,Unnamed: 0,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex,TransactionType
0,0,2019-12-01 00:00:01+00:00,rfQLYTZk7iBSekEv1GughC1ruWfCDeZXVz,rP1afBEfikTz7hJh2ExCDni9W4Bx1dUMRk,16305,30450221009B3E7024448E56F1E580F4AFCBBC9BECF050...,222000000,51752123,Payment
1,1,2019-12-01 00:00:01+00:00,rAPERVgXZavGgiGv6xBgtiZurirW2yAmY,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12645,3045022100C1839ADD174EF85C785E452A83CF0AD655BF...,32,51752123,Payment
2,2,2019-12-01 00:00:02+00:00,rw2ciyaNshpHe7bCHo4bRWq6pqqynnWKQg,rLNaPoKeeBjZe2qs6x52yVPZpZ8td4dc6w,40,3045022100BE2F8FBCC8A76A31CDA4FF0A13BA2120EB71...,360980050,51752124,Payment
3,3,2019-12-01 00:00:10+00:00,rAPERVgXZavGgiGv6xBgtiZurirW2yAmY,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12,3045022100D41EF19727BECD8B77976FC731F86A6F4CD1...,30,51752125,Payment
4,4,2019-12-01 00:00:11+00:00,rAPERVgXZavGgiGv6xBgtiZurirW2yAmY,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12,3045022100B38D74883970A9A21497A520BA4A56C28EF0...,30,51752126,Payment
...,...,...,...,...,...,...,...,...,...
707188,707188,2019-12-30 23:59:42+00:00,rDsbeomae4FXwgQTJp9Rs64Qg9vDiTCdBv,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,60000,304402204918F22DEB8B151FA28FA64AA994B1392B3320...,325686594,52409894,Payment
707189,707189,2019-12-30 23:59:42+00:00,rDsbeomae4FXwgQTJp9Rs64Qg9vDiTCdBv,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,60000,304502210087CC7A6B27090ED095E8BA8F479EF628695C...,275267296,52409894,Payment
707190,707190,2019-12-30 23:59:50+00:00,rDsbeomae4FXwgQTJp9Rs64Qg9vDiTCdBv,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,60000,304402200A23218923346C7E5C868B6941700630A6FBF6...,5083632552,52409895,Payment
707191,707191,2019-12-30 23:59:50+00:00,rDsbeomae4FXwgQTJp9Rs64Qg9vDiTCdBv,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,60000,3044022075F91DAF97CC80563D673B9DFB742DCF73F782...,732509247,52409895,Payment


In [2]:
start_date='2019-12-1'
end_date='2019-12-31'

In [3]:
cred_path='../../credential/Xpring Dev Sandbox-edb2a98acf17.json'

In [4]:
def get_gcp_creds(key_path):
    """
    Return GCP credentials from local .json
    
    Parameters
    ----------
    key_path: str
        Path to local credential .json
    """
    try:
        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes = ["https://www.googleapis.com/auth/cloud-platform"]
        )
    except:
        credentials = None
    return credentials;

In [5]:
def gbq_query(query,cred, query_params=None):
    """
    Run a query against Google Big Query, returning a pandas dataframe of the result.

    Parameters
    ----------
    query: str
        The query string
    cred: obj
        Credential class instance
    
    query_params: list, optional
        The query parameters to pass into the query string
    """
    client = bigquery.Client(credentials=cred)
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    return client.query(query, job_config=job_config).to_dataframe()

In [6]:
#Query and query_parameters
query1 = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex,t.TransactionType
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"

AND TIMESTAMP(l.CloseTime) >=TIMESTAMP(@start_date)
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP(@end_date)


ORDER BY TimeStamp;

"""
query_params1 = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
    bigquery.ScalarQueryParameter("end_date", "STRING", end_date)
]




In [ ]:
xrp1=gbq_query(query1,get_gcp_creds(cred_path),query_params1)

In [ ]:
xrp1